# <div align="center" style="color: #ff5733;">KRI Report New Format</div>

## *Requestor is Jennyfer*

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# FPD30 January 2024 Month End

In [2]:
sq = """SELECT t1.new_loan_type,
ROUND(SUM(CASE when t1.defFPD30 = 1 then 1 ELSE 0 end )/sum(t1.obsFPD30)*100,3) as percentage_of_FPD30
from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
inner JOIN prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core t2
ON
t1.loanAccountNumber = t2.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-01-31')
GROUP BY 1;"""

janfpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
janfpd30

Job ID 939f45fd-88a1-4596-b97e-1a1fa2f95bbd successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,percentage_of_FPD30
0,Quick,14.770
1,SIL-Instore,5.875
2,Flex,3.114


# FPD30 February 2024 Month End

In [3]:
sq = """SELECT t1.new_loan_type,
ROUND(SUM(CASE when t1.defFPD30 = 1 then 1 ELSE 0 end )/sum(t1.obsFPD30)*100,3) as percentage_of_FPD30
from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
inner JOIN prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core t2
ON
t1.loanAccountNumber = t2.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-02-29')
GROUP BY 1;"""

febfpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
febfpd30

Job ID e1cb77ad-97fe-4503-a2eb-e594e31f4026 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,percentage_of_FPD30
0,Quick,13.294
1,SIL-Instore,5.828
2,Flex,2.627


# FPD30 March 2024 Month End

In [4]:
sq = """SELECT t1.new_loan_type,
ROUND(SUM(CASE when t1.defFPD30 = 1 then 1 ELSE 0 end )/sum(t1.obsFPD30)*100,3) as percentage_of_FPD30
from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
inner JOIN prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core t2
ON
t1.loanAccountNumber = t2.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-03-31')
GROUP BY 1;"""

marchfpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
marchfpd30

Job ID ade2f43d-ee17-4afa-9e47-1895adc059e5 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,percentage_of_FPD30
0,Quick,14.429
1,SIL-Instore,6.240
2,Flex,2.700


# 30+, 60+, 90+ for January 2024

In [5]:
sq = """with b as
(select t1.new_loan_type 
, count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-01-31')
group by 1
)
select 
new_loan_type,
round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) thirtyplus,
round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) sixtyplus,
round(sum(nintyplus)/sum(obsnintyplus)*100, 3) nintyplus,
from b
group by 1
order by 1
;"""

secondpartjan24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
secondpartjan24

Job ID 42e4b28b-65ad-48fb-83e0-d302f8ff9abe successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,thirtyplus,sixtyplus,nintyplus
0,Flex,31.579,27.463,24.748
1,Quick,52.124,47.184,46.150
2,SIL-Instore,16.230,12.602,18.935


# 30+, 60+, 90+ for February 2024

In [6]:
sq = """with b as
(select t1.new_loan_type 
, count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-02-29')
group by 1
)
select 
new_loan_type,
round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) thirtyplus,
round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) sixtyplus,
round(sum(nintyplus)/sum(obsnintyplus)*100, 3) nintyplus,
from b
group by 1
order by 1
;"""

secondpartfeb24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
secondpartfeb24

Job ID 6d67378b-f1cb-4c8f-aafb-1bf08bd7cf5e successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,thirtyplus,sixtyplus,nintyplus
0,Flex,29.625,26.069,23.126
1,Quick,51.675,46.800,43.976
2,SIL-Instore,14.710,10.279,16.796


# 30+, 60+, 90+ for March 2024

In [7]:
sq = """with b as
(select t1.new_loan_type 
, count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-03-31')
group by 1
)
select 
new_loan_type,
round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) thirtyplus,
round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) sixtyplus,
round(sum(nintyplus)/sum(obsnintyplus)*100, 3) nintyplus,
from b
group by 1
order by 1
;"""

secondpartmarch24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
secondpartmarch24

Job ID d58a10dc-6999-46ea-8c4b-376e79b13a1a successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,thirtyplus,sixtyplus,nintyplus
0,Flex,33.267,29.034,27.110
1,Quick,58.553,53.424,51.604
2,SIL-Instore,16.344,11.561,16.460


In [8]:
dfjan = janfpd30.merge(secondpartjan24, on='new_loan_type', how='left')
dfjan

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,Quick,14.770,52.124,47.184,46.150
1,SIL-Instore,5.875,16.230,12.602,18.935
2,Flex,3.114,31.579,27.463,24.748


In [9]:
dffeb = febfpd30.merge(secondpartfeb24, on='new_loan_type', how='left')
dffeb

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,Quick,13.294,51.675,46.800,43.976
1,SIL-Instore,5.828,14.710,10.279,16.796
2,Flex,2.627,29.625,26.069,23.126


In [10]:
dfmarch = marchfpd30.merge(secondpartmarch24, on='new_loan_type', how='left')
dfmarch

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,Quick,14.429,58.553,53.424,51.604
1,SIL-Instore,6.240,16.344,11.561,16.460
2,Flex,2.700,33.267,29.034,27.110


In [14]:
# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(r'D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Jennifer Zaldivar\KRI_New_Report\Data\KRI_Report_New.xlsx', engine='xlsxwriter')

# Write each dataframe to a separate worksheet
dfjan.to_excel(writer, sheet_name='January2024')
dffeb.to_excel(writer, sheet_name='February2024')
dfmarch.to_excel(writer, sheet_name='March2024')

# Close the Pandas Excel writer and output the Excel file
writer.close()